In [1]:
!rm -r spaan
!rm -r NERVE
!git clone git://github.com/nicolagulmini/spaan
!git clone git://github.com/nicolagulmini/NERVE
!python -m pip install git+https://github.com/nicolagulmini/tmhmm.py
!pip install Bio

rm: cannot remove 'spaan': No such file or directory
rm: cannot remove 'NERVE': No such file or directory
Cloning into 'spaan'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 195 (delta 109), reused 91 (delta 37), pack-reused 0
Receiving objects: 100% (195/195), 5.63 MiB | 11.38 MiB/s, done.
Resolving deltas: 100% (109/109), done.
Cloning into 'NERVE'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 0), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (10/10), done.
  Cloning https://github.com/nicolagulmini/tmhmm.py to /tmp/pip-req-build-2xx06z6s
  Running command git clone -q https://github.com/nicolagulmini/tmhmm.py /tmp/pip-req-build-2xx06z6s
  Created wheel for tmhmm.py: filename=tmhmm.py-1.3.1-cp37-cp37m-linux_x86_64.whl size=160098 sha256=09511bf3143189efb

In [2]:
import tmhmm
from Bio import SeqIO
from NERVE import protein
from tensorflow import keras
from spaan.data_processing import *

In [3]:
p_ad_no_citoplasm_filter = 0.46
p_ad_extracellular_filter = 0.38
transmemb_dom_limit_filter = 2
e_value = 1e-10
similarity_function = 0.8
verbose = 1

In [4]:
# programma principale
list_of_fasta_proteins = list(SeqIO.parse("./NERVE/data/tre.fasta", "fasta"))

list_of_proteins = []
for p in list_of_fasta_proteins:
	p_id = p.id
	p_seq = p.seq
	list_of_proteins.append(protein.protein(p_id, p_seq))

if verbose > 0:
	for p in list_of_proteins:
		p.print_information()

Information about protein sp|P06846|EBGR_ECOLI:
   accession number = P06846
   length = 327
   localization = None
   estimated probability to be an adhesin = 0
   number of transmembrane domains = None
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None
Information about protein sp|P0AAC4|YBHL_ECOLI:
   accession number = P0AAC4
   length = 234
   localization = None
   estimated probability to be an adhesin = 0
   number of transmembrane domains = None
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None
Information about protein sp|P0AE16|AMPG_ECOLI:
   accession number = P0AE16
   length = 491
   localization = None
   estimated probability to be an adhesin = 0
   number of transmembrane domains = None
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None


In [5]:
# pstorb
# ... cerca di capire come chiamare psortb da qua
# supponendo che ci sia l'output faccio il parsing di ogni final prediction
# final predictions parsing from the output of psortb program

In [6]:
# espaan

model = keras.models.load_model('./spaan/model_trained_on_original_dataset.h5') # insert the path to location!
for p in list_of_proteins:
	p.p_ad = float(model.predict([
			     np.array([aminoacids_frequencies(p.sequence)]),
			     np.array([multiplet_frequencies(p.sequence, 3)]),
			     np.array([multiplet_frequencies(p.sequence, 4)]),
			     np.array([multiplet_frequencies(p.sequence, 5)]),
			     np.array([dipeptide_frequencies(p.sequence)]),
			     np.array([charge_composition(p.sequence)]),
			     np.array([hydrophobic_composition(p.sequence)])
		     ]))

In [7]:
if verbose > 0:
	for p in list_of_proteins:
		p.print_information()

Information about protein sp|P06846|EBGR_ECOLI:
   accession number = P06846
   length = 327
   localization = None
   estimated probability to be an adhesin = 0.01151728630065918
   number of transmembrane domains = None
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None
Information about protein sp|P0AAC4|YBHL_ECOLI:
   accession number = P0AAC4
   length = 234
   localization = None
   estimated probability to be an adhesin = 0.04193472862243652
   number of transmembrane domains = None
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None
Information about protein sp|P0AE16|AMPG_ECOLI:
   accession number = P0AE16
   length = 491
   localization = None
   estimated probability to be an adhesin = 0.02877509593963623
   number of transmembrane domains = None
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None


In [14]:
# tmhmm

for p in list_of_proteins:
    annotation, _ = tmhmm.predict(p.sequence)
    p.tmhmm_seq = annotation
    transmembrane_domains = 0
    for i in range(len(annotation)-1):
        if (annotation[i] == 'i' or annotation[i] == 'o') and annotation[i+1] == 'M':
            transmembrane_domains += 1
    p.transmembrane_doms = transmembrane_domains

/usr/local/lib/python3.7/dist-packages/tmhmm/api.py:20: RuntimeWarning: divide by zero encountered in log
  _, path = viterbi(sequence, *model)


In [15]:
if verbose > 0:
	for p in list_of_proteins:
		p.print_information()

Information about protein sp|P06846|EBGR_ECOLI:
   accession number = P06846
   length = 327
   localization = None
   estimated probability to be an adhesin = 0.01151728630065918
   number of transmembrane domains = 0
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None
Information about protein sp|P0AAC4|YBHL_ECOLI:
   accession number = P0AAC4
   length = 234
   localization = None
   estimated probability to be an adhesin = 0.04193472862243652
   number of transmembrane domains = 7
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None
Information about protein sp|P0AE16|AMPG_ECOLI:
   accession number = P0AE16
   length = 491
   localization = None
   estimated probability to be an adhesin = 0.02877509593963623
   number of transmembrane domains = 14
   number of peptides shared with sapiens = None
   number of peptides shared with mhcpep = None


In [17]:
print(annotation)

iiiiiiiiiiiiMMMMMMMMMMMMMMMMMMMMMMMooooooooooooooMMMMMMMMMMMMMMMMMMMMiiiiiiiiiiiiMMMMMMMMMMMMMMMMMMMMMMMooooMMMMMMMMMMMMMMMMMMMMMMMiiiiiiiiiiiiMMMMMMMMMMMMMMMMMMMMMMMoooooooooMMMMMMMMMMMMMMMMMMMMiiiiiiiiiiiiiiiiiiiiiiiiiiiMMMMMMMMMMMMMMMMMMMMMMMooooooooooooooMMMMMMMMMMMMMMMMMMMMMMMiiiiMMMMMMMMMMMMMMMMMMMMMMMoooooooooMMMMMMMMMMMMMMMMMMMMMMMiiiiiiiiiiiiMMMMMMMMMMMMMMMMMMMMMMMoooMMMMMMMMMMMMMMMMMMMMMMMiiiiiiiiiiiiiiiiiiiMMMMMMMMMMMMMMMMMMMMMMMooooooooooooooMMMMMMMMMMMMMMMMMMMMMMMiiiiiiiiii
